# Search Algorithms

## Introduction

In this project we are going to program an AI snake solver which playes the famous snake game using different search algorithms like BFS,DLS and Astar.In the end we are going to compare time complexity of these algorithms on 3 test sets.

There is snake in a 2D space which the snake's size and width and height of the space are given.There are some specific apples in the board with different scores.If an apple's score is 2 it means that the snake needs to eat 2 apples in that location.The snake has to eat all those apples in the most optimal way.<br>
The snake can only change direction in 4 ways which are up,down,left and right.Other than that,it should not hit its own body.<br>
Whenever the snake its an apple it will grow in length by one.<br>
There are no obstacles in the space and when the snake reaches the walls it can pass through them and come out from the opposite side of the board.

We have 3 test cases and in each of them the first line indicates width and height of the board, the second line indicates the initial location of the snake, the third is the number of apples and the remaining are the location and score of each apple.

In [3]:
import csv
import operator
import copy
from queue import LifoQueue
from collections import deque
import math
import time
import cProfile
import operator
import bisect

from heapq import heappush, heappop


In [4]:
LEFT =(0,-1)
RIGHT =(0,1)
UP =(1,0)
DOWN =(-1,0)

snake_loc_x, snake_loc_y = 0, 0
width, height = 0, 0

First step is to open test sets :<br>
The below function will open each test and return required variables for search functions.

In [283]:
def open_allocate(name):
    apples =()
    with open(name, 'r') as f:
        reader = csv.reader(f)
        count = 1
        for row in reader:
            if count ==1:
                width , height = int(row[0]),int(row[1])
            elif count == 2:
                snake_loc_x , snake_loc_y = int(row[0]),int(row[1])
            elif count == 3:
                num_apples = int(row[0])
            else:
                value = int(row[2])
                while value>0:
                    apples =apples +((int(row[0]),int(row[1])),)
                    value -=1
            count +=1
    return width,height,snake_loc_x,snake_loc_y,apples


In [284]:
width_1,height_1,snake_loc_x_1,snake_loc_y_1,apples_1 = open_allocate('test1.txt')
width_2,height_2,snake_loc_x_2,snake_loc_y_2,apples_2 = open_allocate('test2.txt')
width_3,height_3,snake_loc_x_3,snake_loc_y_3,apples_3 = open_allocate('test3.txt')


Now it is time to model the problem:<br>
At first I was going to use two classes for node and state but then I figured that this aproach is not good enough because calling those classes takes a lot of time.<br>
Then I used tuples to model states and lists for nodes.Each state is 1.the body of snake in the form of tuple of tuples which each tuple is a location in the board ,and 2.apples in the board which is as well tuple of tuples in a way that if there are two apples in one location that location is repeated in my tuple.<br>
I did not used sets because order mattered in the body of snake ,and there might be more than one apple in apples ,therefore using sets was impossible.Lists were not good either because of their complexity when copying and assigning.<br>
Action is movment of snake and eating or not eating an apple.four directions are assigned above which are UP,DOWN,LEFT and RIGHT. first by adding two tuples next location will be found and added to snake's body.since the snake can come out of the other side of the board the remaining of the calculated value is the real location.Then, snake will move and if there is an apple it will be eaten and removed from apples' tuple and if not tail of snake will be cut.<br>
Initial state is first loction of snake and all apples in the board.The final state is when no apple is left in the board meaning that apples' tuple is empty.<br>

### BFS

Here are the steps for the BFS algorithm:<br>

1.Pick a node and enqueue all its adjacent nodes into a queue.<br>

2.Dequeue a node from the queue, mark it as visited and enqueue all its adjacent nodes into a queue.<br>

3.Repeat this process until the queue is empty or you meet a goal.<br>

The program can be stuck in an infinite loop if a node is revisited and was not marked as visited before. Hence, prevent exploring nodes that are visited by marking them as visited.

since in each iteration there is a search in  explored,set data structure is used which has the average of O(1) for x in s.Since poping and adding to queue happens a lot deque is used for frontier to make sure it is a queue.It will return the final node which conists of snakes body and parent of the node which can be used to print the path.

In [352]:
different =0 
num_node =0
def BFS(initial):
    global different 
    global num_node 
    node =(initial,None)
    explored = {initial}
    frontier = deque()
    frontier.append(node)
    while len(frontier) != 0:
        current_node = frontier.popleft()
        current_state = current_node[0]
        body = current_state[0]
        apples =current_state[1]
        head = body[-1]
        #print(head)
        explored.add(current_state)
        different +=1
        for direction in (LEFT,RIGHT,UP,DOWN):
            num_node +=1
            x=head[0]+direction[0]
            y=head[1]+direction[1]
            x= x%width 
            y= y%height
            neighbor = (x,y)
            if neighbor not in body:
                apples_new = apples
                body_new = body + (neighbor,)
                if neighbor in apples_new  : 
                    apples_l=list(apples_new)
                    apples_l.remove(neighbor)
                    apples_new =(tuple(apples_l))
                else:
                    body_l=list(body_new)
                    #print(body_l)
                    del body_l[0]
                    #print(body_l)
                    body_new =(tuple(body_l))
                    #print(body_new)
                child_state =(body_new,apples_new)
                child = [child_state,current_node]
                if len(apples_new)==0:
                        return child
                if child_state not in explored : 
                    frontier.append(child)

Below is the result of BFS for each test.As it is clear first two test are successful in the given time but last one is not.

In [353]:
apples =apples_1
width , height = width_1,height_1
start_pos = (((snake_loc_x_1,snake_loc_y_1),),apples)
node_start = [ start_pos,  None]
tic =time.time()
result =BFS(start_pos)
toc = time.time()
toc-tic

0.3730032444000244

In [354]:
print(different)
different =0 
print(num_node)
num_node =0

18697
74785


In [356]:
apples =apples_2
width , height = width_2,height_2
start_pos = (((snake_loc_x_2,snake_loc_y_2),),apples)
node_start = [ start_pos,  None]
tic =time.time()
result =BFS(start_pos)
toc = time.time()
toc-tic

5.668881416320801

In [357]:
print(different)
different =0 
print(num_node)
num_node =0

308087
1232348


In [349]:
while result!=None:
    x=result[0]
    print(x[0])
    result =result[1]

((7, 10), (6, 10), (6, 9), (6, 8), (6, 7), (6, 6), (5, 6))
((7, 10), (6, 10), (6, 9), (6, 8), (6, 7), (6, 6))
((8, 10), (7, 10), (6, 10), (6, 9), (6, 8), (6, 7))
((9, 10), (8, 10), (7, 10), (6, 10), (6, 9), (6, 8))
((0, 10), (9, 10), (8, 10), (7, 10), (6, 10), (6, 9))
((0, 11), (0, 10), (9, 10), (8, 10), (7, 10), (6, 10))
((0, 11), (0, 10), (9, 10), (8, 10), (7, 10))
((9, 11), (0, 11), (0, 10), (9, 10), (8, 10))
((9, 0), (9, 11), (0, 11), (0, 10), (9, 10))
((9, 1), (9, 0), (9, 11), (0, 11), (0, 10))
((0, 1), (9, 1), (9, 0), (9, 11), (0, 11))
((0, 1), (9, 1), (9, 0), (9, 11))
((0, 1), (9, 1), (9, 0))
((0, 1), (9, 1))
((0, 0), (0, 1))
((0, 0),)


In [181]:
apples =apples_3
width , height = width_3,height_3
start_pos = (((snake_loc_x_3,snake_loc_y_3),),apples)
node_start = [ start_pos,  None]
cProfile.run('BFS(start_pos)')


         27998439 function calls in 42.569 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1   39.176   39.176   42.569   42.569 <ipython-input-177-22f5cc3b7b90>:1(BFS)
        1    0.000    0.000   42.569   42.569 <string>:1(<module>)
        1    0.000    0.000   42.569   42.569 {built-in method builtins.exec}
 13715675    0.990    0.000    0.990    0.000 {built-in method builtins.len}
  3477021    1.531    0.000    1.531    0.000 {method 'add' of 'set' objects}
  6962041    0.494    0.000    0.494    0.000 {method 'append' of 'collections.deque' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
  3477021    0.286    0.000    0.286    0.000 {method 'popleft' of 'collections.deque' objects}
   366677    0.092    0.000    0.092    0.000 {method 'remove' of 'list' objects}




KeyboardInterrupt: 

### IDS

DFS first traverses nodes going through one adjacent of root, then next adjacent. By adding a limit to the depth that our search recurses down the tree DFS turns to DLS. IDS calls DLS for different depths starting from an initial value. In every call, DLS is restricted from going beyond given depth.

To make coming back to each node after expanding it, it is implemented as a recursive function.a global variable is used to store result and the functions only return true or false to stop searching with them as soon as possible.

In [370]:
goal = None
def IDDFS(node ,depth):
    global goal
    for i in range(depth):
        global explored
        explored = set()
        check = DLS(node,i)
        if check == True:
            print(i)
            break 

POS, PARENT = range(2)
                        
def DLS(node,depth):
    global goal
    global different
    global num_node
    pos = node[POS]
    body = pos[0]
    apples =pos[1]
    head = body[-1]
    explored.add(pos)
    different +=1
    if ( depth > 0 ):
        if len(apples)==0 or goal != None:
            return True
        else:
            for direction in (LEFT,RIGHT,UP,DOWN):
                num_node +=1
                x=head[0]+direction[0]
                y=head[1]+direction[1]
                x= x%width 
                y= y%height
                neighbor = (x,y)
                if neighbor not in body:
                    apples_new = apples
                    body_new = body+(neighbor,)
                    if neighbor in apples_new  : 
                        apples_l=list(apples_new)
                        apples_l.remove(neighbor)
                        apples_new =(tuple(apples_l))
                    else:
                        body_l=list(body_new)
                        body_l.pop(0)
                        body_new =(tuple(body_l))
                    child_pos =(body_new,apples_new)
                    child = [child_pos,node]
                    if len(apples_new)==0:
                        goal =child
                        return True
                    else:
                        if child_pos not in explored:  
                            DLS(child, depth-1)

Below are the results of calling IDDFS for each test.

In [371]:
apples =apples_1
width , height = width_1,height_1
start_pos = (((snake_loc_x_1,snake_loc_y_1),),apples)
node_start = [ start_pos,  None]
tic =time.time()
IDDFS(node_start,300)
toc = time.time()
toc-tic

19


0.05285906791687012

In [364]:
print(different)
different =0 
print(num_node)
num_node =0

7329
12917


In [365]:
while goal!=None:
    x=goal[0]
    print(x[0])
    goal =goal[1]

((1, 3), (0, 3), (4, 3), (4, 2), (3, 2), (3, 1))
((1, 3), (0, 3), (4, 3), (4, 2), (3, 2))
((1, 3), (0, 3), (4, 3), (4, 2))
((1, 4), (1, 3), (0, 3), (4, 3))
((1, 4), (1, 3), (0, 3))
((1, 0), (1, 4), (1, 3))
((1, 1), (1, 0), (1, 4))
((1, 1), (1, 0))
((1, 2), (1, 1))
((1, 3), (1, 2))
((1, 4), (1, 3))
((1, 0), (1, 4))
((1, 0),)
((1, 1),)
((0, 1),)
((0, 2),)
((0, 3),)
((0, 4),)
((0, 0),)


In [373]:
goal = None

In [374]:
apples =apples_2
width , height = width_2,height_2
start_pos = (((snake_loc_x_2,snake_loc_y_2),),apples)
node_start = [ start_pos,  None]
tic =time.time()
IDDFS(node_start,300)
toc = time.time()
toc-tic

25


0.9026787281036377

In [368]:
print(different)
different =0 
print(num_node)
num_node =0


101798
207912


In [369]:
while goal!=None:
    x=goal[0]
    print(x[0])
    goal =goal[1]

((7, 10), (6, 10), (6, 9), (6, 8), (6, 7), (6, 6), (5, 6))
((7, 10), (6, 10), (6, 9), (6, 8), (6, 7), (6, 6))
((8, 10), (7, 10), (6, 10), (6, 9), (6, 8), (6, 7))
((9, 10), (8, 10), (7, 10), (6, 10), (6, 9), (6, 8))
((9, 11), (9, 10), (8, 10), (7, 10), (6, 10), (6, 9))
((9, 0), (9, 11), (9, 10), (8, 10), (7, 10), (6, 10))
((9, 0), (9, 11), (9, 10), (8, 10), (7, 10))
((9, 1), (9, 0), (9, 11), (9, 10), (8, 10))
((0, 1), (9, 1), (9, 0), (9, 11), (9, 10))
((0, 2), (0, 1), (9, 1), (9, 0), (9, 11))
((0, 2), (0, 1), (9, 1), (9, 0))
((0, 2), (0, 1), (9, 1))
((0, 3), (0, 2), (0, 1))
((0, 3), (0, 2))
((0, 4), (0, 3))
((0, 5), (0, 4))
((0, 6), (0, 5))
((0, 7), (0, 6))
((0, 8), (0, 7))
((0, 9), (0, 8))
((0, 10), (0, 9))
((0, 11), (0, 10))
((0, 0), (0, 11))
((0, 0),)


In [375]:
goal =None

In [376]:
apples =apples_3
width , height = width_3,height_3
start_pos = (((snake_loc_x_3,snake_loc_y_3),),apples)
node_start = [ start_pos,  None]
tic =time.time()
IDDFS(node_start,300)
toc = time.time()
toc-tic

68


3.592398166656494

In [377]:
print(different)
different =0 
print(num_node)
num_node =0
while goal!=None:
    x=goal[0]
    print(x[0])
    goal =goal[1]

532685
1116054
((3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (4, 8), (4, 7), (5, 7))
((3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (4, 8), (4, 7))
((3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (4, 8))
((3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8))
((3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7))
((3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6))
((4, 0), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5))
((4, 9), (4, 0), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4))
((4, 9), (4, 0), (3, 0), (3, 1), (3, 2), (3, 3))
((3, 9), (4, 9), (4, 0), (3, 0), (3, 1), (3, 2))
((2, 9), (3, 9), (4, 9), (4, 0), (3, 0), (3, 1))
((2, 0), (2, 9), (3, 9), (4, 9), (4, 0), (3, 0))
((2, 1), (2, 0), (2, 9), (3, 9), (4, 9), (4, 0))
((2, 2), (2, 1), (2, 0), (2, 9), (3, 9), (4, 9))
((2, 2), (2, 1), (2, 0), (2, 9), (3, 9))
((2, 3), (2, 2), (2, 1), (2, 0), (2, 9))
((3, 3), (2, 3), (2, 2), (2, 1), (2, 0))
((3, 4), (3, 3), (2, 3), (2, 2), (2, 1))
((4, 4), (3, 4), (3, 3), (2, 3), (2, 2))
((4, 4), (3, 4), (3, 3

### A*

In A start each node has an estimated cost to get to final state.Nodes will go to list in a way that the first one has the least expected cost.The heuristic used in A star is a user defined function which estimates the distance from a given point to the destination.<br>
I am using two distance function for heuristic manhatan and eulicean.In each heuristic total distance of snake's head to all apples is returened based on one of the distance functions. 

In [286]:
def return_manhattan_distance(coord1, coord2):
    x1, y1 = coord1
    x2, y2 = coord2
    return float(abs(x2-x1) + abs(y2-y1))

def return_euclidean_distance(coord1, coord2):
    x1, y1 = coord1
    x2, y2 = coord2
    x_dist_sq = math.pow((x2-x1), 2)
    y_dist_sq = math.pow((y2-y1), 2)
    return math.sqrt(x_dist_sq + y_dist_sq)

def heuristic_eu(pos):
    apples = pos[1]
    body = pos[0]
    head = body[-1]
    heu =0
    for apple in apples:
        x,y =apple[0],apple[1]
        heu += return_euclidean_distance((x,y),head)
    return heu

def heuristic_man(pos):
    apples = pos[1]
    body = pos[0]
    head = body[-1]
    heu =0
    for apple in apples:
        x,y =apple[0],apple[1]
        heu += return_manhattan_distance((x,y),head)
    return heu

Both heuristics are admissible and consistent because real distance is less than a straight line or going along axes and real distance is at any time more than or equal to expected distance.

Since nodes have to be sorted, heap is used to handle them (adding and poping) because its more optimal in comparison to other data structures.In this algorithm each node is consist of seven values in a list.first three values are f,h ,and g function values of the node then it is the state(coded as POS) and finally it comes to OPEN indicating wether we have seen the node previously or not, VALID showing that there is a better path to this node by being false ,and parent node.

In [419]:
F, H, G, POS, OPEN, VALID, PARENT = range(7)


def astar(start_pos, neighbors, goal, start_g, heuristic,alpha):
    global different
    global num_node
    start_h = heuristic(start_pos)
    start = [ start_g + (start_h*alpha),start_h, start_g, start_pos, True,True, None]

    # Track all nodes seen so far.
    nodes = {start_pos: start}
    # Maintain a heap of nodes.
    heap = [start]

    # Track the best path found so far.
    best = start

    while heap:
        current = heappop(heap)
        current[OPEN] = False
        # Have we reached the goal?
        if goal(current[POS]):
            best = current
            break
        different +=1
        # Visit the neighbors of the current node.
        for neighbor_pos in neighbors(current[POS]):
            num_node +=1
            neighbor_g = current[G] +1
            neighbor = nodes.get(neighbor_pos)
            if neighbor is None:
                # We have found a new node.
                neighbor_h = heuristic(neighbor_pos)
                neighbor = [neighbor_g + (neighbor_h*alpha), neighbor_h, neighbor_g, neighbor_pos, True, True, current]
                nodes[neighbor_pos] = neighbor
                heappush(heap, neighbor)
                if neighbor_h < best[H]:
                    best = neighbor

            elif neighbor_g < neighbor[G]:

                # We have found a better path to the neighbor.
                if neighbor[OPEN]:

                    neighbor[VALID] = False
                    nodes[neighbor_pos] = neighbor = neighbor[:]
                    neighbor[F] = neighbor_g + (neighbor[H]*alpha)
                    neighbor[G] = neighbor_g
                    neighbor[VALID] = True
                    neighbor[PARENT] = current
                    heappush(heap, neighbor)

                else:

                    # Reopen the neighbor.
                    neighbor[F] = neighbor_g + (neighbor[H]*alpha)
                    neighbor[G] = neighbor_g
                    neighbor[PARENT] = current
                    neighbor[OPEN] = True
                    heappush(heap, neighbor)
        while heap and not heap[0][VALID]:
            heappop(heap)
            
    return best

I have commented each step in the function above.Just for it to be more understandable I seperated neighbor functon and goal function.neighbors return four locatons in the board where snake can go.goal is returning wether we have reached final or not.

In [393]:
def neighbors(pos):
    body = pos[0]
    apples =pos[1]
    head = body[-1]
    for direction in (LEFT,RIGHT,UP,DOWN):
        x=head[0]+direction[0]
        y=head[1]+direction[1]
        x= x%width 
        y= y%height
        neighbor = (x,y)
        if neighbor not in body:
            apples_new = apples
            body_new = body+(neighbor,)
            if neighbor in apples_new  : 
                apples_l=list(apples_new)
                apples_l.remove(neighbor)
                apples_new =(tuple(apples_l))
            else:
                body_l=list(body_new)
                body_l.pop(0)
                body_new =(tuple(body_l))
            yield (body_new,apples_new)
        
def goal(pos):
    apples = pos[1]
    if len(apples)==0:
        return True
    else:
        return False

By passing one as alpha we have simple A* which its result are as below.

In [396]:
apples =apples_1
width , height = width_1,height_1
start_pos = (((snake_loc_x_1,snake_loc_y_1),),apples)
node_start = [ start_pos,  None]
tic =time.time()
best =astar(start_pos, neighbors, goal,0, heuristic_eu,1)
toc = time.time()
toc-tic

0.000997304916381836

In [397]:
print(different)
different =0 
print(num_node)
num_node =0
while best[PARENT]!=None:
    state =best[POS]
    print(state[0])
    best = best[PARENT]

50
146
((3, 1), (2, 1), (2, 2), (2, 3), (1, 3), (1, 4))
((3, 1), (2, 1), (2, 2), (2, 3), (1, 3))
((3, 2), (3, 1), (2, 1), (2, 2), (2, 3))
((4, 2), (3, 2), (3, 1), (2, 1), (2, 2))
((4, 3), (4, 2), (3, 2), (3, 1), (2, 1))
((3, 3), (4, 3), (4, 2), (3, 2), (3, 1))
((3, 3), (4, 3), (4, 2), (3, 2))
((3, 3), (4, 3), (4, 2))
((2, 3), (3, 3), (4, 3))
((2, 3), (3, 3))
((2, 4), (2, 3))
((1, 4), (2, 4))
((0, 4), (1, 4))
((0, 4),)


In [297]:
apples =apples_1
width , height = width_1,height_1
start_pos = (((snake_loc_x_1,snake_loc_y_1),),apples)
node_start = [ start_pos,  None]
tic =time.time()
best =astar(start_pos, neighbors, goal,0, heuristic_man,1)
toc = time.time()
toc-tic

0.0009982585906982422

In [298]:
best[POS]

(((3, 1), (2, 1), (1, 1), (1, 2), (1, 3), (1, 4)), ())

In [303]:
apples =apples_2
width , height = width_2,height_2
start_pos = (((snake_loc_x_2,snake_loc_y_2),),apples)
node_start = [ start_pos,  None]
tic =time.time()
best =astar(start_pos, neighbors, goal,0, heuristic_man,1)
toc = time.time()
toc-tic

0.0059850215911865234

In [304]:
best[POS]

(((1, 6), (0, 6), (0, 5), (0, 4), (0, 3), (0, 2), (0, 1)), ())

In [398]:
apples =apples_2
width , height = width_2,height_2
start_pos = (((snake_loc_x_2,snake_loc_y_2),),apples)
node_start = [ start_pos,  None]
tic =time.time()
best =astar(start_pos, neighbors, goal,0, heuristic_eu,1)
toc = time.time()
toc-tic

0.003988504409790039

In [399]:
print(different)
different =0 
print(num_node)
num_node =0
while best[PARENT]!=None:
    state =best[POS]
    print(state[0])
    best = best[PARENT]

73
218
((6, 6), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (6, 10))
((6, 6), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10))
((7, 6), (6, 6), (5, 6), (5, 7), (5, 8), (5, 9))
((7, 5), (7, 6), (6, 6), (5, 6), (5, 7), (5, 8))
((8, 5), (7, 5), (7, 6), (6, 6), (5, 6), (5, 7))
((8, 4), (8, 5), (7, 5), (7, 6), (6, 6), (5, 6))
((8, 4), (8, 5), (7, 5), (7, 6), (6, 6))
((9, 4), (8, 4), (8, 5), (7, 5), (7, 6))
((9, 3), (9, 4), (8, 4), (8, 5), (7, 5))
((9, 2), (9, 3), (9, 4), (8, 4), (8, 5))
((0, 2), (9, 2), (9, 3), (9, 4), (8, 4))
((0, 1), (0, 2), (9, 2), (9, 3), (9, 4))
((9, 1), (0, 1), (0, 2), (9, 2), (9, 3))
((9, 0), (9, 1), (0, 1), (0, 2), (9, 2))
((9, 11), (9, 0), (9, 1), (0, 1), (0, 2))
((0, 11), (9, 11), (9, 0), (9, 1), (0, 1))
((0, 11), (9, 11), (9, 0), (9, 1))
((0, 0), (0, 11), (9, 11), (9, 0))
((0, 0), (0, 11), (9, 11))
((0, 0), (0, 11))


In [306]:
apples =apples_3
width , height = width_3,height_3
start_pos = (((snake_loc_x_3,snake_loc_y_3),),apples)
node_start = [ start_pos,  None]
tic =time.time()
best =astar(start_pos, neighbors, goal,0, heuristic_man,1)
toc = time.time()
toc-tic

0.02689361572265625

In [307]:
best[POS]

(((6, 5), (6, 4), (6, 3), (6, 2), (6, 1), (7, 1), (8, 1), (9, 1)), ())

In [400]:
apples =apples_3
width , height = width_3,height_3
start_pos = (((snake_loc_x_3,snake_loc_y_3),),apples)
node_start = [ start_pos,  None]
tic =time.time()
best =astar(start_pos, neighbors, goal,0, heuristic_eu,1)
toc = time.time()
toc-tic

0.07880115509033203

In [401]:
print(different)
different =0 
print(num_node)
num_node =0
while best[PARENT]!=None:
    state =best[POS]
    print(state[0])
    best = best[PARENT]

1441
4039
((3, 2), (4, 2), (5, 2), (6, 2), (7, 2), (8, 2), (8, 1), (9, 1))
((3, 2), (4, 2), (5, 2), (6, 2), (7, 2), (8, 2), (8, 1))
((3, 3), (3, 2), (4, 2), (5, 2), (6, 2), (7, 2), (8, 2))
((3, 4), (3, 3), (3, 2), (4, 2), (5, 2), (6, 2), (7, 2))
((3, 5), (3, 4), (3, 3), (3, 2), (4, 2), (5, 2), (6, 2))
((3, 6), (3, 5), (3, 4), (3, 3), (3, 2), (4, 2), (5, 2))
((4, 6), (3, 6), (3, 5), (3, 4), (3, 3), (3, 2), (4, 2))
((5, 6), (4, 6), (3, 6), (3, 5), (3, 4), (3, 3), (3, 2))
((5, 7), (5, 6), (4, 6), (3, 6), (3, 5), (3, 4), (3, 3))
((4, 7), (5, 7), (5, 6), (4, 6), (3, 6), (3, 5), (3, 4))
((4, 7), (5, 7), (5, 6), (4, 6), (3, 6), (3, 5))
((4, 8), (4, 7), (5, 7), (5, 6), (4, 6), (3, 6))
((4, 9), (4, 8), (4, 7), (5, 7), (5, 6), (4, 6))
((5, 9), (4, 9), (4, 8), (4, 7), (5, 7), (5, 6))
((5, 8), (5, 9), (4, 9), (4, 8), (4, 7), (5, 7))
((5, 8), (5, 9), (4, 9), (4, 8), (4, 7))
((5, 7), (5, 8), (5, 9), (4, 9), (4, 8))
((4, 7), (5, 7), (5, 8), (5, 9), (4, 9))
((4, 7), (5, 7), (5, 8), (5, 9))
((3, 7), (4

Below are weighted A* by eulicean heuristic and alpha as 2 and 4.

In [402]:
apples =apples_1
width , height = width_1,height_1
start_pos = (((snake_loc_x_1,snake_loc_y_1),),apples)
node_start = [ start_pos,  None]
tic =time.time()
best =astar(start_pos, neighbors, goal,0, heuristic_eu,4)
toc = time.time()
toc-tic

0.0009968280792236328

In [403]:
print(different)
different =0 
print(num_node)
num_node =0
while best[PARENT]!=None:
    state =best[POS]
    print(state[0])
    best = best[PARENT]

19
58
((1, 3), (1, 4), (1, 0), (2, 0), (2, 1), (3, 1))
((1, 3), (1, 4), (1, 0), (2, 0), (2, 1))
((0, 3), (1, 3), (1, 4), (1, 0), (2, 0))
((4, 3), (0, 3), (1, 3), (1, 4), (1, 0))
((4, 2), (4, 3), (0, 3), (1, 3), (1, 4))
((4, 2), (4, 3), (0, 3), (1, 3))
((3, 2), (4, 2), (4, 3), (0, 3))
((3, 3), (3, 2), (4, 2), (4, 3))
((3, 3), (3, 2), (4, 2))
((2, 3), (3, 3), (3, 2))
((2, 3), (3, 3))
((2, 4), (2, 3))
((1, 4), (2, 4))
((0, 4), (1, 4))
((0, 4),)


In [421]:
apples =apples_1
width , height = width_1,height_1
start_pos = (((snake_loc_x_1,snake_loc_y_1),),apples)
node_start = [ start_pos,  None]
tic =time.time()
best =astar(start_pos, neighbors, goal,0, heuristic_eu,2)
toc = time.time()
toc-tic

0.0009975433349609375

In [422]:
print(different)
different =0 
print(num_node)
num_node =0
while best[PARENT]!=None:
    state =best[POS]
    print(state[0])
    best = best[PARENT]

59
182
((1, 3), (1, 4), (1, 0), (2, 0), (2, 1), (3, 1))
((1, 3), (1, 4), (1, 0), (2, 0), (2, 1))
((0, 3), (1, 3), (1, 4), (1, 0), (2, 0))
((4, 3), (0, 3), (1, 3), (1, 4), (1, 0))
((3, 3), (4, 3), (0, 3), (1, 3), (1, 4))
((3, 3), (4, 3), (0, 3), (1, 3))
((3, 2), (3, 3), (4, 3), (0, 3))
((2, 2), (3, 2), (3, 3), (4, 3))
((2, 2), (3, 2), (3, 3))
((2, 3), (2, 2), (3, 2))
((2, 3), (2, 2))
((2, 4), (2, 3))
((1, 4), (2, 4))
((0, 4), (1, 4))
((0, 4),)


In [405]:
apples =apples_2
width , height = width_2,height_2
start_pos = (((snake_loc_x_2,snake_loc_y_2),),apples)
node_start = [ start_pos,  None]
tic =time.time()
best =astar(start_pos, neighbors, goal,0, heuristic_eu,2)
toc = time.time()
toc-tic

0.0009963512420654297

In [406]:
print(different)
different =0 
print(num_node)
num_node =0
while best[PARENT]!=None:
    state =best[POS]
    print(state[0])
    best = best[PARENT]

40
118
((6, 6), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (6, 10))
((6, 6), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10))
((7, 6), (6, 6), (5, 6), (5, 7), (5, 8), (5, 9))
((7, 5), (7, 6), (6, 6), (5, 6), (5, 7), (5, 8))
((8, 5), (7, 5), (7, 6), (6, 6), (5, 6), (5, 7))
((8, 4), (8, 5), (7, 5), (7, 6), (6, 6), (5, 6))
((8, 4), (8, 5), (7, 5), (7, 6), (6, 6))
((9, 4), (8, 4), (8, 5), (7, 5), (7, 6))
((9, 3), (9, 4), (8, 4), (8, 5), (7, 5))
((9, 2), (9, 3), (9, 4), (8, 4), (8, 5))
((0, 2), (9, 2), (9, 3), (9, 4), (8, 4))
((0, 1), (0, 2), (9, 2), (9, 3), (9, 4))
((9, 1), (0, 1), (0, 2), (9, 2), (9, 3))
((9, 0), (9, 1), (0, 1), (0, 2), (9, 2))
((9, 11), (9, 0), (9, 1), (0, 1), (0, 2))
((0, 11), (9, 11), (9, 0), (9, 1), (0, 1))
((0, 11), (9, 11), (9, 0), (9, 1))
((0, 0), (0, 11), (9, 11), (9, 0))
((0, 0), (0, 11), (9, 11))
((0, 0), (0, 11))


In [411]:
apples =apples_2
width , height = width_2,height_2
start_pos = (((snake_loc_x_2,snake_loc_y_2),),apples)
node_start = [ start_pos,  None]
tic =time.time()
best =astar(start_pos, neighbors, goal,0, heuristic_eu,4)
toc = time.time()
toc-tic

0.0009975433349609375

In [412]:
print(different)
different =0 
print(num_node)
num_node =0
while best[PARENT]!=None:
    state =best[POS]
    print(state[0])
    best = best[PARENT]

20
59
((6, 6), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (6, 10))
((6, 6), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10))
((7, 6), (6, 6), (5, 6), (5, 7), (5, 8), (5, 9))
((7, 5), (7, 6), (6, 6), (5, 6), (5, 7), (5, 8))
((8, 5), (7, 5), (7, 6), (6, 6), (5, 6), (5, 7))
((8, 4), (8, 5), (7, 5), (7, 6), (6, 6), (5, 6))
((8, 4), (8, 5), (7, 5), (7, 6), (6, 6))
((9, 4), (8, 4), (8, 5), (7, 5), (7, 6))
((9, 3), (9, 4), (8, 4), (8, 5), (7, 5))
((9, 2), (9, 3), (9, 4), (8, 4), (8, 5))
((0, 2), (9, 2), (9, 3), (9, 4), (8, 4))
((0, 1), (0, 2), (9, 2), (9, 3), (9, 4))
((9, 1), (0, 1), (0, 2), (9, 2), (9, 3))
((9, 0), (9, 1), (0, 1), (0, 2), (9, 2))
((9, 11), (9, 0), (9, 1), (0, 1), (0, 2))
((0, 11), (9, 11), (9, 0), (9, 1), (0, 1))
((0, 11), (9, 11), (9, 0), (9, 1))
((0, 0), (0, 11), (9, 11), (9, 0))
((0, 0), (0, 11), (9, 11))
((0, 0), (0, 11))


In [413]:
apples =apples_3
width , height = width_3,height_3
start_pos = (((snake_loc_x_3,snake_loc_y_3),),apples)
node_start = [ start_pos,  None]
tic =time.time()
best =astar(start_pos, neighbors, goal,0, heuristic_eu,2)
toc = time.time()
toc-tic

0.018947362899780273

In [414]:
print(different)
different =0 
print(num_node)
num_node =0
while best[PARENT]!=None:
    state =best[POS]
    print(state[0])
    best = best[PARENT]

579
1626
((4, 9), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0), (8, 1), (9, 1))
((4, 9), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0), (8, 1))
((4, 8), (4, 9), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0))
((5, 8), (4, 8), (4, 9), (4, 0), (5, 0), (6, 0), (7, 0))
((5, 7), (5, 8), (4, 8), (4, 9), (4, 0), (5, 0), (6, 0))
((4, 7), (5, 7), (5, 8), (4, 8), (4, 9), (4, 0), (5, 0))
((4, 6), (4, 7), (5, 7), (5, 8), (4, 8), (4, 9), (4, 0))
((4, 5), (4, 6), (4, 7), (5, 7), (5, 8), (4, 8), (4, 9))
((4, 5), (4, 6), (4, 7), (5, 7), (5, 8), (4, 8))
((4, 4), (4, 5), (4, 6), (4, 7), (5, 7), (5, 8))
((3, 4), (4, 4), (4, 5), (4, 6), (4, 7), (5, 7))
((3, 4), (4, 4), (4, 5), (4, 6), (4, 7))
((3, 5), (3, 4), (4, 4), (4, 5), (4, 6))
((3, 6), (3, 5), (3, 4), (4, 4), (4, 5))
((3, 7), (3, 6), (3, 5), (3, 4), (4, 4))
((4, 7), (3, 7), (3, 6), (3, 5), (3, 4))
((4, 7), (3, 7), (3, 6), (3, 5))
((5, 7), (4, 7), (3, 7), (3, 6))
((5, 6), (5, 7), (4, 7), (3, 7))
((4, 6), (5, 6), (5, 7), (4, 7))
((4, 5), (4, 6), (5, 6), (5, 7))
((4, 5), (

In [416]:
apples =apples_3
width , height = width_3,height_3
start_pos = (((snake_loc_x_3,snake_loc_y_3),),apples)
node_start = [ start_pos,  None]
tic =time.time()
best =astar(start_pos, neighbors, goal,0, heuristic_eu,4)
toc = time.time()
toc-tic

0.0019948482513427734

In [417]:
print(different)
different =0 
print(num_node)
num_node =0
while best[PARENT]!=None:
    state =best[POS]
    print(state[0])
    best = best[PARENT]

110
322
((4, 9), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0), (8, 1), (9, 1))
((4, 9), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0), (8, 1))
((4, 8), (4, 9), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0))
((4, 7), (4, 8), (4, 9), (4, 0), (5, 0), (6, 0), (7, 0))
((4, 6), (4, 7), (4, 8), (4, 9), (4, 0), (5, 0), (6, 0))
((3, 6), (4, 6), (4, 7), (4, 8), (4, 9), (4, 0), (5, 0))
((3, 5), (3, 6), (4, 6), (4, 7), (4, 8), (4, 9), (4, 0))
((3, 4), (3, 5), (3, 6), (4, 6), (4, 7), (4, 8), (4, 9))
((3, 4), (3, 5), (3, 6), (4, 6), (4, 7), (4, 8))
((4, 4), (3, 4), (3, 5), (3, 6), (4, 6), (4, 7))
((4, 5), (4, 4), (3, 4), (3, 5), (3, 6), (4, 6))
((5, 5), (4, 5), (4, 4), (3, 4), (3, 5), (3, 6))
((5, 6), (5, 5), (4, 5), (4, 4), (3, 4), (3, 5))
((5, 7), (5, 6), (5, 5), (4, 5), (4, 4), (3, 4))
((5, 7), (5, 6), (5, 5), (4, 5), (4, 4))
((4, 7), (5, 7), (5, 6), (5, 5), (4, 5))
((4, 6), (4, 7), (5, 7), (5, 6), (5, 5))
((4, 5), (4, 6), (4, 7), (5, 7), (5, 6))
((5, 5), (4, 5), (4, 6), (4, 7), (5, 7))
((5, 5), (4, 5), (4, 6), (4, 

### Conclusion

###### Test1

| Algorithm | Distance | Path| Visited States| Different Visited States| Time
| --- | --- | --- | --- | --- | --- 
| BFS | 12 | DRDRULDLLLDD |74785|18697|0.21
| DLS | 19 | DDDDRDDDDDDDDLDLL |7329|12917|0.03
| A* | 14 | DRRDRRDLDLUULU |146|50|0.001
| Weighted A*  alpha=2| 15 | DRRDDRURRRUURUR |182|59|0.0009
| Weighted A*  alpha=4| 15 | DRRDRDRURRUURUR |58|19|0.0009

###### Test2

| Algorithm | Distance | Path| Visited States| Different Visited States| Time
| --- | --- | --- | --- | --- | --- 
| BFS | 15 | ULDDRDLLLLDDDDL |1232348|308087|0.5
| DLS | 25 | DDDDDDDDDDDLDDDLLLDDDDL |207912|101798|0.6
| A* | 20 | DLUURULUULULULLUUUUR |218|73|0.0019
| Weighted A*  alpha=2| 20 | DLUURULUULULULLUUUUR |118|40|0.0009
| Weighted A*  alpha=4| 20 | DLUURULUULULULLUUUUR |59|20|0.0009

###### Test3

| Algorithm | Distance | Path| Visited States| Different Visited States| Time
| --- | --- | --- | --- | --- | --- 
| BFS | - | - |-|-|-
| DLS | 68 | DDDDDDDDDDLLDDDDDDDDDLDDDDDDDDDDLDDLDDDDDLLDDLDDDRRULUUUUUUUURDR |1116054|532685|10
| A* | 32 | URURURUUUURRUULDDRDLLDDDDRRRRRDR |4039|1441|0.07
| Weighted A*  alpha=2| 32 | URURURUURURULLDDDRUUURULUURRRRUR |1626|549|0.016
| Weighted A*  alpha=4| 36 | URURURUURUURDDLUURDDLDLUURUUUURRRRUR |322|110|0.002

In all  tests higher alpha results into less visited states and time.It's impact ,however, is more evident in more complicated test cases(test3 for instance).Overall A*'s visited states and time is better than the other two algorithms.among them DLS seems to be better in time since its using both BFS and DFS 's benefits.